<a href="https://colab.research.google.com/github/sandwrath/NLP/blob/main/Dash_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install jupyter-dash

In [ ]:
!pip install dash_bootstrap_components

In [3]:
!pip install dash_html_components

In [4]:
!pip install dash_core_components

In [ ]:
!pip install plotly==4.5.2

In [6]:
import pathlib
import re
import json
from datetime import datetime
import flask
import dash
import dash_table
import matplotlib.colors as mcolors
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import numpy as np
from dash.dependencies import Output, Input, State
from dateutil import relativedelta
from wordcloud import WordCloud, STOPWORDS
from sklearn.manifold import TSNE
from jupyter_dash import JupyterDash

In [7]:
JupyterDash.infer_jupyter_proxy_config()

In [8]:
#PLOTLY_LOGO = "https://images.plot.ly/logo/new-branding/plotly-logomark.png"
GLOBAL_DF = pd.read_csv('/content/drive/MyDrive/ABSA/Final Data/Globaldata.csv',index_col= 0)



In [9]:
df = pd.read_csv('/content/drive/MyDrive/ABSA/Final Data/test.csv',index_col= 0)

In [10]:
df.head()

,Aspect,Aspect-term,sentiment,id
0,GUI,platform,2,1
1,Download,year,0,1
2,Pricing,month,0,1
3,Improvement,money,0,1
4,Download,suppport,2,1


In [11]:
tmp_df = pd.read_csv('/content/drive/MyDrive/ABSA/Final Data/reviews.csv')

In [12]:
tmp = GLOBAL_DF['Date received']

In [13]:
GLOBAL_DF.drop(GLOBAL_DF.index, inplace=True)

In [14]:
GLOBAL_DF.Company = df.Aspect

In [15]:
GLOBAL_DF['Consumer complaint narrative'] = df['Aspect-term']

In [16]:
GLOBAL_DF['Date received'] = tmp

In [17]:
GLOBAL_DF = GLOBAL_DF.dropna(axis=1, how='all')

In [18]:

GLOBAL_DF["Date received"] = pd.to_datetime(
    GLOBAL_DF["Date received"]
)


In [19]:
GLOBAL_DF

,Date received,Consumer complaint narrative,Company
0,2017-04-30,platform,GUI
1,2016-05-23,year,Download
2,2015-03-25,month,Pricing
3,2016-06-21,money,Improvement
4,2016-06-04,suppport,Download
...,...,...,...
386,2016-09-29,nothing,GUI
387,2016-02-03,help,Other
388,2017-02-07,screen,GUI
389,2017-04-06,task,GUI


In [20]:
ADDITIONAL_STOPWORDS = [
    "XXXX",
    "XX",
    "xx",
    "xxxx",
    "n't",
    "Trans Union",
    "BOA",
    "Citi",
    "account",
]

In [21]:
for stopword in ADDITIONAL_STOPWORDS:
    STOPWORDS.add(stopword)


In [22]:

def sample_data(dataframe, float_percent):
    """
    Returns a subset of the provided dataframe.
    The sampling is evenly distributed and reproducible
    """
    print("making a local_df data sample with float_percent: %s" % (float_percent))
    return dataframe.sample(frac=float_percent, random_state=1)


In [23]:

def get_complaint_count_by_company(dataframe):
    """ Helper function to get complaint counts for unique Applications """
    company_counts = dataframe["Company"].value_counts()
    # we filter out all Applications with less than 11 complaints for now
    company_counts = company_counts[company_counts > 10]
    values = company_counts.keys().tolist()
    counts = company_counts.tolist()
    return values, counts


In [24]:

def calculate_Application_sample_data(dataframe, sample_size, time_values):
    """ TODO """
    print(
        "making Application_sample_data with sample_size count: %s and time_values: %s"
        % (sample_size, time_values)
    )
    if time_values is not None:
        min_date = time_values[0]
        max_date = time_values[1]
        dataframe = dataframe[
            (dataframe["Date received"] >= min_date)
            & (dataframe["Date received"] <= max_date)
        ]
    company_counts = dataframe["Company"].value_counts()
    company_counts_sample = company_counts[:sample_size]
    values_sample = company_counts_sample.keys().tolist()
    counts_sample = company_counts_sample.tolist()

    return values_sample, counts_sample


In [25]:

def make_local_df(selected_Application, time_values, n_selection):
    """ TODO """
    print("redrawing Application-wordcloud...")
    n_float = float(n_selection / 100)
    print("got time window:", str(time_values))
    print("got n_selection:", str(n_selection), str(n_float))
    # sample the dataset according to the slider
    local_df = sample_data(GLOBAL_DF, n_float)
    if time_values is not None:
        time_values = time_slider_to_date(time_values)
        local_df = local_df[
            (local_df["Date received"] >= time_values[0])
            & (local_df["Date received"] <= time_values[1])
        ]
    if selected_Application:
        local_df = local_df[local_df["Company"] == selected_Application]
    return local_df


In [26]:
def make_marks_time_slider(mini, maxi):
    """
    A helper function to generate a dictionary that should look something like:
    {1420066800: '2015', 1427839200: 'Q2', 1435701600: 'Q3', 1443650400: 'Q4',
    1451602800: '2016', 1459461600: 'Q2', 1467324000: 'Q3', 1475272800: 'Q4',
     1483225200: '2017', 1490997600: 'Q2', 1498860000: 'Q3', 1506808800: 'Q4'}
    """
    step = relativedelta.relativedelta(months=+1)
    start = datetime(year=mini.year, month=1, day=1)
    end = datetime(year=maxi.year, month=maxi.month, day=30)
    ret = {}

    current = start
    while current <= end:
        current_str = int(current.timestamp())
        if current.month == 1:
            ret[current_str] = {
                "label": str(current.year),
                "style": {"font-weight": "bold"},
            }
        elif current.month == 4:
            ret[current_str] = {
                "label": "Q2",
                "style": {"font-weight": "lighter", "font-size": 7},
            }
        elif current.month == 7:
            ret[current_str] = {
                "label": "Q3",
                "style": {"font-weight": "lighter", "font-size": 7},
            }
        elif current.month == 10:
            ret[current_str] = {
                "label": "Q4",
                "style": {"font-weight": "lighter", "font-size": 7},
            }
        else:
            pass
        current += step
    # print(ret)
    return ret


In [27]:

def time_slider_to_date(time_values):
    """ TODO """
    min_date = datetime.fromtimestamp(time_values[0]).strftime("%c")
    max_date = datetime.fromtimestamp(time_values[1]).strftime("%c")
    print("Converted time_values: ")
    print("\tmin_date:", time_values[0], "to: ", min_date)
    print("\tmax_date:", time_values[1], "to: ", max_date)
    return [min_date, max_date]


In [28]:

def make_options_Application_drop(values):
    """
    Helper function to generate the data format the dropdown dash component wants
    """
    ret = []
    for value in values:
        ret.append({"label": value, "value": value})
    return ret


In [29]:
def plotly_wordcloud(data_frame):
    """A wonderful function that returns figure data for three equally
    wonderful plots: wordcloud, frequency histogram and treemap"""
    complaints_text = list(data_frame["Consumer complaint narrative"].dropna().values)

    if len(complaints_text) < 1:
        return {}, {}, {}

    # join all documents in corpus
    text = " ".join(list(complaints_text))

    word_cloud = WordCloud(stopwords=set(STOPWORDS), max_words=100, max_font_size=90)
    word_cloud.generate(text)

    word_list = []
    freq_list = []
    fontsize_list = []
    position_list = []
    orientation_list = []
    color_list = []

    for (word, freq), fontsize, position, orientation, color in word_cloud.layout_:
        word_list.append(word)
        freq_list.append(freq)
        fontsize_list.append(fontsize)
        position_list.append(position)
        orientation_list.append(orientation)
        color_list.append(color)

    # get the positions
    x_arr = []
    y_arr = []
    for i in position_list:
        x_arr.append(i[0])
        y_arr.append(i[1])

    # get the relative occurence frequencies
    new_freq_list = []
    for i in freq_list:
        new_freq_list.append(i * 80)

    trace = go.Scatter(
        x=x_arr,
        y=y_arr,
        textfont=dict(size=new_freq_list, color=color_list),
        hoverinfo="text",
        textposition="top center",
        hovertext=["{0} - {1}".format(w, f) for w, f in zip(word_list, freq_list)],
        mode="text",
        text=word_list,
    )

    layout = go.Layout(
        {
            "xaxis": {
                "showgrid": False,
                "showticklabels": False,
                "zeroline": False,
                "automargin": True,
                "range": [-100, 250],
            },
            "yaxis": {
                "showgrid": False,
                "showticklabels": False,
                "zeroline": False,
                "automargin": True,
                "range": [-100, 450],
            },
            "margin": dict(t=20, b=20, l=10, r=10, pad=4),
            "hovermode": "closest",
        }
    )

    wordcloud_figure_data = {"data": [trace], "layout": layout}
    word_list_top = word_list[:25]
    word_list_top.reverse()
    freq_list_top = freq_list[:25]
    freq_list_top.reverse()

    frequency_figure_data = {
        "data": [
            {
                "y": word_list_top,
                "x": freq_list_top,
                "type": "bar",
                "name": "",
                "orientation": "h",
            }
        ],
        "layout": {"height": "550", "margin": dict(t=20, b=20, l=100, r=20, pad=4)},
    }
    treemap_trace = go.Treemap(
        labels=word_list_top, parents=[""] * len(word_list_top), values=freq_list_top
    )
    treemap_layout = go.Layout({"margin": dict(t=10, b=10, l=5, r=5, pad=4)})
    treemap_figure = {"data": [treemap_trace], "layout": treemap_layout}
    return wordcloud_figure_data, frequency_figure_data, treemap_figure


In [30]:

NAVBAR = dbc.Navbar(
    children=[
        html.A(
            # Use row and col to control vertical alignment of logo / brand
            dbc.Row(
                [
                    #dbc.Col(html.Img(src=PLOTLY_LOGO, height="30px")),
                    dbc.Col(
                        dbc.NavbarBrand("Application Reviews", className="ml-2")
                    ),
                ],
                align="center",
                no_gutters=True,
            ),
            href="https://plot.ly",
        )
    ],
    color="dark",
    dark=True,
    sticky="top",
)

LEFT_COLUMN = dbc.Jumbotron(
    [
        html.H4(children="Select Application's Aspect & dataset size", className="display-5"),
        html.Hr(className="my-2"),
        html.Label("Select percentage of dataset", className="lead"),
        html.P(
            "(Lower is faster. Higher is more precise)",
            style={"fontSize": 10, "font-weight": "lighter"},
        ),
        dcc.Slider(
            id="n-selection-slider",
            min=1,
            max=100,
            step=1,
            marks={
                0: "0%",
                10: "",
                20: "20%",
                30: "",
                40: "40%",
                50: "",
                60: "60%",
                70: "",
                80: "80%",
                90: "",
                100: "100%",
            },
            value=20,
        ),
        html.Label("Select a Application's Aspect", style={"marginTop": 50}, className="lead"),
        html.P(
            "(You can use the dropdown or click the barchart on the right)",
            style={"fontSize": 10, "font-weight": "lighter"},
        ),
        dcc.Dropdown(
            id="Application-drop", clearable=False, style={"marginBottom": 50, "font-size": 12}
        ),
        html.Label("Select time frame", className="lead"),
        html.Div(dcc.RangeSlider(id="time-window-slider"), style={"marginBottom": 50}),
        html.P(
            "(You can define the time frame down to month granularity)",
            style={"fontSize": 10, "font-weight": "lighter"},
        ),
    ]
)


In [31]:
df = df.replace([0, 1, 2], ['pos', 'neg', 'pos'])

In [32]:
df_fig1 = df.groupby(by=["Aspect","sentiment"]).count()

In [33]:
df_fig1 = df_fig1.reset_index()

In [34]:
aspect_fig = px.bar(df_fig1, x="Aspect", y='Aspect-term', color="sentiment", barmode="group")

In [35]:
sunBurst_fig = px.sunburst(df_fig1, path=['Aspect','sentiment'], values='Aspect-term',color='sentiment')

In [36]:
WORDCLOUD_PLOTS = [
    dbc.CardHeader(html.H5("Most frequently used words in reviews")),
    dbc.Alert(
        "Not enough data to render these plots, please adjust the filters",
        id="no-data-alert",
        color="warning",
        style={"display": "none"},
    ),
    dbc.CardBody(
        [
            dbc.Row(
                [
                    dbc.Col(
                        dcc.Loading(
                            id="loading-frequencies",
                            children=[dcc.Graph(id="frequency_figure")],
                            type="default",
                        )
                    ),
                    dbc.Col(
                        [
                            dcc.Tabs(
                                id="tabs",
                                children=[
                                    dcc.Tab(
                                        label="Treemap",
                                        children=[
                                            dcc.Loading(
                                                id="loading-treemap",
                                                children=[dcc.Graph(id="Application-treemap")],
                                                type="default",
                                            )
                                        ],
                                    ),
                                    dcc.Tab(
                                        label="Wordcloud",
                                        children=[
                                            dcc.Loading(
                                                id="loading-wordcloud",
                                                children=[
                                                    dcc.Graph(id="Application-wordcloud")
                                                ],
                                                type="default",
                                            )
                                        ],
                                    ),
                                ],
                            )
                        ],
                        md=8,
                    ),
                ]
            )
        ]
    ),
]


TOP_ApplicationS_PLOT = [
    dbc.CardHeader(html.H5("Aspects by number of complaints")),
    dbc.CardBody(
        [
            dcc.Loading(
                id="loading-Applications-hist",
                children=[
                    dbc.Alert(
                        "Not enough data to render this plot, please adjust the filters",
                        id="no-data-alert-Application",
                        color="warning",
                        style={"display": "none"},
                    ),
                    dcc.Graph(id="Application-sample"),
                ],
                type="default",
            )
        ],
        style={"marginTop": 0, "marginBottom": 0},
    ),
]


ASPECT_Bar_PLOT = [
               dbc.CardHeader(html.H5("The Aspects and the thier Sentiment")),
               dbc.CardBody(
                   [
                    dcc.Graph(id='Aspect-Bar',figure=aspect_fig)
                   ],
                   style={"marginTop": 0, "marginBottom": 0,},
               )
               
]

ASPECT_Sunburst_PLOT = [
               dbc.CardHeader(html.H5("The Aspects and the thier Sentiment")),
               dbc.CardBody(
                   [
                    dcc.Graph(id='Aspect-sunburst',figure=sunBurst_fig)
                   ],
                   style={"marginTop": 0, "marginBottom": 0,},
               )
               
]




In [37]:
BODY = dbc.Container(
    [     
     dbc.Card(ASPECT_Bar_PLOT, style={"marginTop": 30},),    
     dbc.Row(
      [
          dbc.Col(LEFT_COLUMN, md=4, align="center"),
          dbc.Col(dbc.Card(TOP_ApplicationS_PLOT), md=8),
      ],
      style={"marginTop": 30},
     ),
     dbc.Card(WORDCLOUD_PLOTS),
     dbc.Card(ASPECT_Sunburst_PLOT, style={"margin": 30},),
     
    ],
    className="mt-12",
)



In [38]:

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

server = app.server

app.layout = html.Div(children=[NAVBAR, BODY])

In [39]:

@app.callback(Output("Application-drop", "value"), [Input("Application-sample", "clickData")])
def update_Application_drop_on_click(value):
    """ TODO """
    if value is not None:
        selected_Application = value["points"][0]["x"]
        return selected_Application
    return "App"


In [40]:

@app.callback(
    [
        Output("time-window-slider", "marks"),
        Output("time-window-slider", "min"),
        Output("time-window-slider", "max"),
        Output("time-window-slider", "step"),
        Output("time-window-slider", "value"),
    ],
    [Input("n-selection-slider", "value")],
)
def populate_time_slider(value):
    """
    Depending on our dataset, we need to populate the time-slider
    with different ranges. This function does that and returns the
    needed data to the time-window-slider.
    """
    value += 0
    min_date = GLOBAL_DF["Date received"].min()
    max_date = GLOBAL_DF["Date received"].max()

    marks = make_marks_time_slider(min_date, max_date)
    min_epoch = list(marks.keys())[0]
    max_epoch = list(marks.keys())[-1]

    return (
        marks,
        min_epoch,
        max_epoch,
        (max_epoch - min_epoch) / (len(list(marks.keys())) * 3),
        [min_epoch, max_epoch],
    )


@app.callback(
    Output("Application-drop", "options"),
    [Input("time-window-slider", "value"), Input("n-selection-slider", "value")],
)
def populate_Application_dropdown(time_values, n_value):
    """ TODO """
    print("Application-drop: TODO USE THE TIME VALUES AND N-SLIDER TO LIMIT THE DATASET")
    if time_values is not None:
        pass
    n_value += 1
    Application_names, counts = get_complaint_count_by_company(GLOBAL_DF)
    counts.append(1)
    return make_options_Application_drop(Application_names)


@app.callback(
    [Output("Application-sample", "figure"), Output("no-data-alert-Application", "style")],
    [Input("n-selection-slider", "value"), Input("time-window-slider", "value")],
)
def update_Application_sample_plot(n_value, time_values):
    """ TODO """
    print("redrawing Application-sample...")
    print("\tn is:", n_value)
    print("\ttime_values is:", time_values)
    if time_values is None:
        return [{}, {"display": "block"}]
    n_float = float(n_value / 100)
    Application_sample_count = 10
    local_df = sample_data(GLOBAL_DF, n_float)
    min_date, max_date = time_slider_to_date(time_values)
    values_sample, counts_sample = calculate_Application_sample_data(
        local_df, Application_sample_count, [min_date, max_date]
    )
    data = [
        {
            "x": values_sample,
            "y": counts_sample,
            "text": values_sample,
            "textposition": "auto",
            "type": "bar",
            "name": "",
        }
    ]
    layout = {
        "autosize": False,
        "margin": dict(t=10, b=10, l=40, r=0, pad=4),
        "xaxis": {"showticklabels": False},
    }
    print("redrawing Application-sample...done")
    return [{"data": data, "layout": layout}, {"display": "none"}]


In [41]:
@app.callback(
    [
        Output("Application-wordcloud", "figure"),
        Output("frequency_figure", "figure"),
        Output("Application-treemap", "figure"),
        Output("no-data-alert", "style"),
    ],
    [
        Input("Application-drop", "value"),
        Input("time-window-slider", "value"),
        Input("n-selection-slider", "value"),
    ],
)
def update_wordcloud_plot(value_drop, time_values, n_selection):
    """ Callback to rerender wordcloud plot """
    local_df = make_local_df(value_drop, time_values, n_selection)
    wordcloud, frequency_figure, treemap = plotly_wordcloud(local_df)
    alert_style = {"display": "none"}
    if (wordcloud == {}) or (frequency_figure == {}) or (treemap == {}):
        alert_style = {"display": "block"}
    print("redrawing Application-wordcloud...done")
    return (wordcloud, frequency_figure, treemap, alert_style)


# @app.callback(
#     [
#      Output("Aspect-Bar","figure"),
#     ]
#     [
#         Input("n-selection-slider", "value"),
#     ],
# )
# def update_aspect_bar_plot(n_selection):
#   {
      
#   }

In [42]:
app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>